In [74]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.formula.api as smf
from pymer4.models import Lmer
from pymer4 import stats as pstats
from pymer4.simulate import easy_multivariate_normal
rng = np.random.default_rng()
from sklearn.metrics import roc_curve, roc_auc_score
from matplotlib import pyplot as plt
from IPython.display import display
from sklearn.model_selection import KFold, GroupKFold
from sklearn.linear_model import LinearRegression
from pathlib import Path
import itertools

from fitter import Fitter

import seaborn as sns
%matplotlib inline
pd.set_option("display.max_rows", 100)

In [14]:
class Simmer(object):
    """
    Class for simulating data for complex mixed effects models based on a model and a
    skeleton of the data structure.

    Parameters
    ----------
    model : string
        Lmer compatible model specification string.
    skeleton: Pandas.DataFrame
        Design matrix with all independent variables and grouping variables. The size 
        of this skeleton determines the size of the simulated dataset.
    seed: {None, int, array_like[ints], SeedSequence, BitGenerator, Generator}, optional
        Seed for random number generator. See documentaiton for np.random.default_rng().
    """
    def __init__(self, model, skeleton, seed=None):
        
        self.rng = np.random.default_rng(seed)

        self.model = model
        self.dep = model.split('~')[0].strip()
        skeleton[self.dep] = self.rng.uniform(-1,1, size=skeleton.shape[0])
        self.skeleton = skeleton
        
        # We'll fit the model on random dependent data just to populate all the 
        # variables we need.
        premdl = Lmer(model, data=skeleton)
        preres = premdl.fit()
        self.premdl = premdl
        self.grp_names = list(premdl.grps.keys())
        self.terms = self.premdl.coefs.index.values
        display(premdl.coefs)
        
    def sim_data(self, fixef=None, ranef_stds=None, ranef_corr=None, e_scale=0.2):
        """
        Create a simulated dataset, optionally specifying fixed effects, random effects
        standard deviations, and random effects correlations.
        
        Parameters
        ----------
        fixef : list or function, optional
            List of the mean fixed effects values you would like to simulate. Must have 
            length equal to the number of fixed effects including the intercept if you
            specified one in your model. You may instead pass a function that accepts
            the length of the list as a single parameter and returns a list of values. 
            If nothing is passed random fixed effects will be generated from a uniform
            distribution from -1 to 1. Simmer.premdl.coefs gives the order and length.
        ranef_stds: list or function, optional
            List of the standard deviations of the random effects you would like to 
            simulate. Must have length equal to the number of random effects including.
            You may instead pass a function that accepts the length of the list as a 
            single parameter and returns a list of values. If nothing is passed random 
            random effects will be generated from a uniform distribution from 0 to 1.
            Simmer.premdl.ranef_var gives the order and length.
        ranef_stds: list or function, optional
            List of the correlations between the random effects you would like to 
            simulate. Must have length equal to the number of random effects correlations.
            You may instead pass a function that accepts the length of the list as a 
            single parameter and returns a list of values. If nothing is passed random 
            correlations will be generated from a uniform distribution from -0.5 to 0.5.
            Simmer.premdl.ranef_corr gives the order and length.
        e_scale: float, optional
            Scale of the standard normal distribution used to add noise to each observation.
            
            
        Returns
        -------
        sim_clean: Pandas.DataFrame
            Simulated data in similar shape to skeleton with simulated dependent variable added.
        """
        
        self.e_scale = e_scale
        # deal with fixed effects first
        sim_fixef_names = ['fe' + '__' + nn for nn in self.premdl.coefs.index.values]
        if fixef is None:
            fixef_dat = [list(self.rng.uniform(-1, 1, len(sim_fixef_names)))] * len(self.skeleton)
        else:
            try:
                fixef_dat = [list(fixef(len(sim_fixef_names)))] * len(self.skeleton)
            except TypeError:
                if len(fixef) != len(sim_fixef_names):
                    raise ValueError(f"fixef must have length {len(sim_fixef_names)}, got {len(fixef)} instead.")
                fixef_dat = [list(fixef)] * len(self.skeleton)
        sim_fixefs = pd.DataFrame(data = fixef_dat, columns=sim_fixef_names)
        self.sim_fixefs = sim_fixefs
        
        # now random effects
        sim_ranef_var = self.premdl.ranef_var.copy()
        if ranef_stds is None:
            sim_ranef_var['Std'] = self.rng.uniform(0, 0.5, len(sim_ranef_var))
        else:
            try: 
                sim_ranef_var['Std'] = ranef_stds(len(sim_ranef_var))
            except TypeError:
                if len(ranef_stds) != len(sim_ranef_var):
                    raise ValueError(f"fixef must have length {len(sim_ranef_var)}, got {len(ranef_stds)} instead.")
                sim_ranef_var['Std'] = ranef_stds
        sim_ranef_var['Var'] = sim_ranef_var.Std **2
        sim_ranef_var = sim_ranef_var.reset_index().rename(columns={'index':'level'})
        self.sim_ranef_var = sim_ranef_var
        
        # and random effect correlations
        sim_ranef_corr = self.premdl.ranef_corr.copy()
        if ranef_corr is None:
            sim_ranef_corr['Corr'] = self.rng.uniform(-0.5, 0.5, len(sim_ranef_corr))
        else:
            try: 
                sim_ranef_corr['Corr'] = ranef_corr(len(sim_ranef_corr))
            except TypeError:
                if len(ranef_corr) != len(sim_ranef_corr):
                    raise ValueError(f"fixef must have length {len(sim_ranef_corr)}, got {len(ranef_corr)} instead.")
                sim_ranef_corr['Corr'] = ranef_corr
        sim_ranef_corr = sim_ranef_corr.reset_index().rename(columns={'index':'level'})
        self.sim_ranef_corr = sim_ranef_corr
        
        # now we can build the simulated dataset
        sim = self.skeleton.drop(self.dep, axis=1)
        sim = pd.concat([sim, sim_fixefs], axis=1)
        
        # generate group-wise random effects
        fixed_cors = {}
        blups = self.skeleton.groupby(self.grp_names).first().reset_index().loc[:, self.grp_names]
        for grp in self.grp_names:
            grp_ranef_var = sim_ranef_var.loc[sim_ranef_var.level == grp]
            grp_ranef_corr = sim_ranef_corr.loc[sim_ranef_corr.level == grp]
            nobs = self.skeleton[grp].nunique()
            nfeats = len(grp_ranef_var)
            cor_vars, fc = easy_multivariate_normal(nobs, nfeats,
                                                grp_ranef_corr.Corr.values,
                                                sigma=grp_ranef_var.Std.values,
                                                return_new_corrs=True
                                                )
            fixed_cors[grp] = fc
            cor_var_names = [grp + '__' + nn for nn in grp_ranef_var.Name.values]
            cor_vars = pd.DataFrame(data=cor_vars, columns=cor_var_names)
            cor_vars[grp] = sorted(self.skeleton[grp].unique())
            blups = blups.merge(cor_vars, how='left', on=grp)
        sim = sim.merge(blups, how='left', on=self.grp_names)
        self.fixed_cors = fixed_cors
        
        # deal with interactions
        for term in self.premdl.coefs.index.values:
            if ':' in term:
                sim[term] = sim.loc[:, term.split(':')].product(1)
        
        # add noise term
        sim['noise'] = self.rng.normal(scale=self.e_scale, size=len(sim))
        
        self._sim = sim
        
        # aggregate terms
        aggregated_terms = self._aggregate_terms()

        # sum aggregated terms to create simulated values for dependent variable
        sim[self.dep] = aggregated_terms.sum(1)
        
        self.sim_clean = sim.loc[:, self.skeleton.columns]
        
        return self.sim_clean.copy()
    
    def tweak_term(self, tweaks):
        for term, value in tweaks.items():
            self._sim[term] = value

        # aggregate terms
        aggregated_terms = self._aggregate_terms()

        # sum aggregated terms to create simulated values for dependent variable
        sim = self._sim.copy()
        sim[self.dep] = aggregated_terms.sum(1)
        
        self.sim_clean = sim.loc[:, self.skeleton.columns]
        
        return self.sim_clean.copy()
    
    def _aggregate_terms(self):
        # calculate all the terms of the linear model
        aggregated_terms = pd.DataFrame(columns=self.terms, index=None)
        for term in self.terms:
            if term == '(Intercept)':
                col_matches = []
                for ix, cc in enumerate(self._sim.columns):
                    if '__' in cc:
                        cc = cc.split('__')[-1]
                    if cc == term:
                        col_matches.append(self._sim.columns[ix])
                aggregated_terms[term] = self._sim.loc[:, col_matches].sum(axis=1)
            else:
                col_matches = []
                for ix, cc in enumerate(self._sim.columns):
                    if '__' in cc:
                        cc = cc.split('__')[-1]
                        if cc == term:
                            col_matches.append(self._sim.columns[ix])
                
                summed_res = self._sim.loc[:, col_matches].sum(axis=1)
                aggregated_terms[term] = self._sim[term] * summed_res
        
        # add some noise
        aggregated_terms['noise'] = self._sim['noise']
        
        return aggregated_terms

In [15]:
def kfold_linear(data, model):
    kf = KFold(n_splits=8)
    fits = []
    reses = []
    for fix, (train_ind, test_ind) in enumerate(kf.split(data)):
        mdl = smf.ols(model, data.loc[train_ind])
        fitted = mdl.fit()
        res = data.loc[test_ind, ['SDAN'] + [mdl.endog_names]].copy()
        res[f'{mdl.endog_names}_predicted'] = fitted.predict(data.loc[test_ind])
        res['fold'] = fix
        reses.append(res)
        fits.append(fitted)
    reses = pd.concat(reses)
    return fits, reses

def kfold_lme(data, model, other_vars = None):
    gkf = GroupKFold(n_splits=8)
    fit_mdls = []
    reses = []
    for tix,(train_ind, test_ind) in enumerate(gkf.split(data, groups=data.SDAN)):
        mdl = Lmer(model, data.loc[train_ind])
        _ = mdl.fit()
        y_name = model.split("~")[0].strip()
        vars_to_copy = ['SDAN'] + [y_name]
        if other_vars is not None:
            vars_to_copy.extend(other_vars)
        res = data.loc[test_ind, vars_to_copy].copy()
        res[f'{y_name}_predicted'] = mdl.predict(data.loc[test_ind])
        res['fold'] = tix
        reses.append(res)
        fit_mdls.append(mdl)
    reses = pd.concat(reses)
    return fit_mdls, reses

def get_rsquared(var, df):
    var_pred = var + '_predicted'
    num = ((df[var] - df[var_pred])**2).sum()
    den = ((df[var] - df[var].mean())**2).sum()
    r_squared = 1 - (num / den)
    return r_squared

def get_95ci(var, df):
    means = []
    for inds in itertools.product(df.index, repeat=len(df.index)):
        vals = []
        for ix in inds:
            vals.append(df.loc[ix, var])
        means.append(np.mean(vals))
    lci, uci = np.percentile(means, [2.5, 97.5])
    return pd.Series({f'{var}_lci': lci, f'{var}_uci':uci})

def get_99ci(var, df):
    means = []
    for inds in itertools.product(df.index, repeat=len(df.index)):
        vals = []
        for ix in inds:
            vals.append(df.loc[ix, var])
        means.append(np.mean(vals))
    lci, uci = np.percentile(means, [0.5, 99.5])
    return pd.Series({f'{var}_lci': lci, f'{var}_uci':uci})

def get_999ci(var, df):
    means = []
    bootstrap_folds = list(itertools.product(df.index, repeat=len(df.index)))
    bootstrap_folds = set([tuple(sorted(x)) for x in bootstrap_folds])
    for inds in bootstrap_folds:
        vals = []
        for ix in inds:
            vals.append(df.loc[ix, var])
        means.append(np.mean(vals))
    lci, uci = np.percentile(means, [0.05, 99.95])
    return pd.Series({f'{var}_lci': lci, f'{var}_uci':uci})

# def corr_plot(corr, for_cors, alpha=0.05, bonferoni_denomenator=None):
#     mask = np.tril(np.ones_like(corr, dtype=bool))
#     annots = corr.copy()
#     ps = corr.copy()
#     if bonferoni_denomenator is None:
#         bonferoni_alpha = alpha/mask.sum()
#     else:
#         bonferoni_alpha = alpha/bonferoni_denomenator
#     for ix,row in corr.iterrows():
#         for name in row.index:
#             r, p = stats.pearsonr(for_cors[ix], for_cors[name])
#             ps.loc[ix, name] = p
#             annot = f"${corr.loc[ix, name]:0.2g}"
#             if p < bonferoni_alpha:
#                 annot += "^*$"
#             else:
#                 annot += "$"
#             annots.loc[ix, name] = annot
    
#     fig, ax = plt.subplots(figsize=(11, 9), dpi=250)
#     cmap = sns.diverging_palette(230, 20, as_cmap=True)
#     sns.heatmap(corr.iloc[:-1, 1:], mask=mask[:-1, 1:], cmap=cmap, vmax=1, vmin=-1, center=0,
#                 square=True, linewidths=.5, cbar_kws={"shrink": .5},
#                 annot=annots.iloc[:-1, 1:], fmt="")
#     return fig,ax

def corr_plot(corr, for_cors, alpha=0.05, bonferoni_denomenator=None, cluster_var=None):
    mask = np.tril(np.ones_like(corr, dtype=bool))
    annots = corr.copy()
    ps = corr.copy()
    if bonferoni_denomenator is None:
        bonferoni_alpha = alpha/mask.sum()
    else:
        bonferoni_alpha = alpha/bonferoni_denomenator
    for ix,row in corr.iterrows():
        for name in row.index:
            if cluster_var is None:
                r, p = stats.pearsonr(for_cors[ix], for_cors[name])
            elif ix == name:
                p = np.nan
            else:
                zs = for_cors.loc[for_cors['level_1'] == ix, name]
                zs = zs[pd.notnull(zs)]
                t, p = stats.ttest_1samp(zs, 0)
                mean = np.tanh(for_cors.loc[for_cors['level_1'] == ix, name].mean())
                se = np.tanh(for_cors.loc[for_cors['level_1'] == ix, name].std() / np.sqrt(for_cors[cluster_var].nunique()))
                dof = for_cors[cluster_var].nunique() - 1
            ps.loc[ix, name] = p
            annot = f"${corr.loc[ix, name]:0.2g}"
    #         if cluster_var is not None:
    #             se = np.tanh(for_cors.loc[for_cors['level_1'] == ix, name].std() / np.sqrt(for_cors[cluster_var].nunique()))
    #             annot += f"\pm{se:0.2g}"
            if p < bonferoni_alpha:
                if cluster_var is None:
                    if ix != name:
                        print(ix, name, r, len(for_cors[ix]), p )
                else:
                    print(ix, name, mean, se, t, dof, p)
                annot += "^*$"
            else:
                annot += "$"
            annots.loc[ix, name] = annot
    corr.index.name = None
    fig, ax = plt.subplots(figsize=(11, 9), dpi=250)
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    sns.heatmap(corr.iloc[:-1, 1:], mask=mask[:-1, 1:], cmap=cmap, vmax=1, vmin=-1, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5},
                annot=annots.iloc[:-1, 1:], fmt="")
    return fig,ax

In [20]:
model = 's_mfq_tot ~ dep_immed*TimeBetween + MFQtminus1 + InpatientDuring + Age_at_visit + SEX + antidepressants + OtherMeds + postpandemic + (TimeBetween | SDAN)'
nointeraction = 's_mfq_tot ~ dep_immed + TimeBetween + MFQtminus1 + InpatientDuring + Age_at_visit + SEX + antidepressants + OtherMeds + postpandemic + (TimeBetween | SDAN)'

In [35]:
mdat = pd.read_csv('../data/MFQAnalysesDatabaseforDylan.csv', index_col=0).reset_index(drop=True)
bsl_mfqs = mdat.sort_values(['SDAN', 'Clinical_Visit_Date']).groupby('SDAN')[['MFQtminus1']].first().reset_index().rename(columns={'MFQtminus1':'baseline_mfq'})
mdat = mdat.merge(bsl_mfqs, how='left', on='SDAN')


In [36]:
mdat.groupby('SDAN').Participant_Type2.count().mean(), mdat.groupby('SDAN').Participant_Type2.count().std() 

(10.084615384615384, 6.731891720014073)

In [37]:
mdat['TimeBetween'] = mdat.TimeBetween / 365
mdat['Time2'] = mdat.TimeBetween ** 2
mdat['dep_immed:TimeBetween'] = mdat.dep_immed * mdat.TimeBetween
mdat['TimeBetween:dep_immed'] = mdat.dep_immed * mdat.TimeBetween
mdat['dep_immed:Time2'] = mdat.dep_immed * mdat.TimeBetween
mdat['TimeBetween:MFQtminus1'] = mdat.MFQtminus1 * mdat.TimeBetween
mdat['TimeBetween:baseline_mfq'] = mdat.baseline_mfq * mdat.TimeBetween
mdat['InpatientDuring'] = mdat.InpatientDuring.astype(int)
mdat['SEX'] = (mdat.SEX == 'FEMALE').astype(int)
mdat['postpandemic'] = mdat.postpandemic.astype(int)

In [38]:
real_mdl = Lmer(model, mdat)
real_res = real_mdl.fit()
real_res

Formula: s_mfq_tot~dep_immed*TimeBetween+MFQtminus1+InpatientDuring+Age_at_visit+SEX+antidepressants+OtherMeds+postpandemic+(TimeBetween|SDAN)

Family: gaussian	 Inference: parametric

Number of observations: 1311	 Groups: {'SDAN': 130.0}

Log-likelihood: -3774.841 	 AIC: 7549.682

Random effects:

                 Name     Var    Std
SDAN      (Intercept)   3.269  1.808
SDAN      TimeBetween  13.187  3.631
Residual               16.646  4.080

              IV1          IV2   Corr
SDAN  (Intercept)  TimeBetween -0.482

Fixed effects:



,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),6.156,2.473,9.839,1.879,88.873,3.276,0.002,**
dep_immed,0.672,-0.387,1.731,0.540,69.203,1.244,0.218,
TimeBetween,-0.319,-2.908,2.270,1.321,45.452,-0.242,0.810,
MFQtminus1,0.581,0.538,0.625,0.022,609.526,26.008,0.000,***
InpatientDuring,-0.772,-1.654,0.110,0.450,735.381,-1.716,0.087,.
Age_at_visit,-0.173,-0.390,0.044,0.111,83.467,-1.566,0.121,
SEX,1.332,0.463,2.200,0.443,51.272,3.005,0.004,**
antidepressants,0.011,-0.860,0.882,0.444,52.946,0.024,0.981,
OtherMeds,0.389,-0.579,1.358,0.494,52.259,0.788,0.434,
postpandemic,-0.353,-0.979,0.272,0.319,655.301,-1.106,0.269,


In [39]:
skeleton = mdat.loc[:, ['dep_immed', 'TimeBetween', 'MFQtminus1', 'InpatientDuring', 'Age_at_visit', 'SEX', 'antidepressants', 'OtherMeds', 'postpandemic', 'SDAN']]

In [40]:
# use most of the fixed effects from the real data for the simulated data
fixefs = real_mdl.coefs.Estimate.values
# just switch out the last one
#fixefs[-1] = 0

In [41]:
real_mdl.ranef_corr

,IV1,IV2,Corr
SDAN,(Intercept),TimeBetween,-0.482032


In [52]:
simmer = Simmer(model, skeleton, seed=0)
clean_sim = simmer.sim_data(fixefs, real_mdl.ranef_var.Std, real_mdl.ranef_corr.Corr,  e_scale=4)
mdl = Lmer(model, data=clean_sim)
mdl.fit(old_optimizer=True)

unable to evaluate scaled gradient 

Model failed to converge: degenerate  Hessian with 1 negative eigenvalues 

Formula: s_mfq_tot~dep_immed*TimeBetween+MFQtminus1+InpatientDuring+Age_at_visit+SEX+antidepressants+OtherMeds+postpandemic+(TimeBetween|SDAN)

Family: gaussian	 Inference: parametric

Number of observations: 1311	 Groups: {'SDAN': 130.0}

Log-likelihood: -1154.964 	 AIC: 2309.929

Random effects:

                 Name    Var    Std
SDAN      (Intercept)  0.000  0.000
SDAN      TimeBetween  0.034  0.183
Residual               0.326  0.571

              IV1          IV2   Corr
SDAN  (Intercept)  TimeBetween -0.196

Fixed effects:



,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),-0.215849,-0.553490,0.121791,0.172269,782.399966,-1.252982,0.210587,
dep_immed,0.004531,-0.085478,0.094540,0.045924,971.862743,0.098666,0.921424,
TimeBetween,0.159709,-0.116814,0.436231,0.141086,65.961607,1.132001,0.261733,
MFQtminus1,0.003152,-0.001839,0.008143,0.002547,1062.549651,1.237770,0.216075,
InpatientDuring,0.014126,-0.085331,0.113584,0.050744,1293.977265,0.278382,0.780764,
Age_at_visit,0.010931,-0.008945,0.030808,0.010141,731.634016,1.077900,0.281434,
SEX,0.032004,-0.038869,0.102877,0.036160,640.082827,0.885047,0.376464,
antidepressants,-0.018930,-0.091394,0.053533,0.036972,549.449805,-0.512021,0.608842,
OtherMeds,-0.023286,-0.103950,0.057379,0.041156,556.761604,-0.565784,0.571769,
postpandemic,0.006343,-0.068606,0.081293,0.038240,1233.646630,0.165883,0.868277,


Formula: s_mfq_tot~dep_immed*TimeBetween+MFQtminus1+InpatientDuring+Age_at_visit+SEX+antidepressants+OtherMeds+postpandemic+(TimeBetween|SDAN)

Family: gaussian	 Inference: parametric

Number of observations: 1311	 Groups: {'SDAN': 130.0}

Log-likelihood: -3832.489 	 AIC: 7664.977

Random effects:

                 Name     Var    Std
SDAN      (Intercept)  11.602  3.406
SDAN      TimeBetween  22.119  4.703
Residual               16.511  4.063

              IV1          IV2   Corr
SDAN  (Intercept)  TimeBetween -0.429

Fixed effects:



,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),7.900,2.603,13.198,2.703,237.844,2.923,0.004,**
dep_immed,0.913,-0.687,2.513,0.816,117.113,1.118,0.266,
TimeBetween,-1.194,-4.094,1.706,1.480,77.383,-0.807,0.422,
MFQtminus1,0.583,0.535,0.632,0.025,1246.848,23.488,0.000,***
InpatientDuring,-1.089,-2.037,-0.140,0.484,1104.269,-2.250,0.025,*
Age_at_visit,-0.270,-0.583,0.043,0.160,234.515,-1.690,0.092,.
SEX,0.988,-0.424,2.400,0.720,119.326,1.371,0.173,
antidepressants,0.770,-0.640,2.180,0.719,118.170,1.070,0.287,
OtherMeds,1.275,-0.298,2.848,0.803,116.059,1.589,0.115,
postpandemic,-0.177,-0.898,0.543,0.368,839.099,-0.482,0.630,


In [59]:
models = {
    'Null': 's_mfq_tot ~ antidepressants + TimeBetween + InpatientDuring + Age_at_visit + SEX  + OtherMeds + postpandemic + (TimeBetween | SDAN)',
    'FH': 's_mfq_tot ~ dep_immed*TimeBetween + InpatientDuring + Age_at_visit + SEX + antidepressants + OtherMeds + postpandemic + (TimeBetween | SDAN)',
}

In [64]:
tweaked.columns

Index(['dep_immed', 'TimeBetween', 'MFQtminus1', 'InpatientDuring',
       'Age_at_visit', 'SEX', 'antidepressants', 'OtherMeds', 'postpandemic',
       'SDAN', 's_mfq_tot', 'dep_immed:TimeBetween', 'TimeBetween:dep_immed',
       'dep_immed:Time2', 'TimeBetween:MFQtminus1'],
      dtype='object')

In [68]:
!mkdir ../data/mfq_sims

In [77]:
def foldwise_bootstraps(df, path=None):
    if path is not None:
        path = Path(path)
        if not path.parent.exists():
            raise ValueError(f"If path is not None, the parent directory should exist. {path.parent} does not exist.")
        
    df = df.copy()
    metrics = ['rmse', 'mae', 'rmse_unweighted', 'mae_unweighted']
    res = []
    folds = df.fold.unique()
    model_names = df.model.unique()
    bootstrap_folds = list(itertools.product(folds, repeat=len(folds)))
    bootstrap_folds = set([tuple(sorted(x)) for x in bootstrap_folds])
    for iix, inds in enumerate(bootstrap_folds):
        for bsfi, ix in enumerate(inds):
            for metric in metrics:
                row={}
                row['bsi'] = iix
                row['bsfold'] = bsfi
                row['metric'] = metric
                for model in model_names:
                    row[f'{model}'] = df.loc[(df.model == model) & (df.fold == ix), metric].values[0]
                res.append(row)
    pairwise_res = pd.DataFrame(res)
    if path is not None:
        pairwise_res.to_csv(path, index=None)
    return pairwise_res

In [ ]:
sim_number = 0
pw_reses = []
for fh_tb in [0, 3, 6, 9, 12]:
    tweaked = simmer.tweak_term({'fe__dep_immed:TimeBetween':fh_tb})
    tweaked['dep_immed:TimeBetween'] = tweaked.dep_immed * tweaked.TimeBetween
    tweaked['TimeBetween:dep_immed'] = tweaked.dep_immed * tweaked.TimeBetween
    tweaked['dep_immed:Time2'] = tweaked.dep_immed * tweaked.TimeBetween
    tweaked['TimeBetween:MFQtminus1'] = tweaked.MFQtminus1 * tweaked.TimeBetween
    reses = []
    for mix, model in models.items():
        ffs, rrs = kfold_lme(tweaked, model, other_vars=['TimeBetween'])
        rrs['model'] = mix
        reses.append(rrs)
    reses = pd.concat(reses)
    # now make mf_agg
    var = 's_mfq_tot'
    var_pred = var + '_predicted'
    
    reses['squared_error'] = (reses[var] - reses[var_pred]) ** 2
    reses['residual'] = reses[var] - reses[var_pred]
    reses['abs_error'] = np.abs(reses[var] - reses[var_pred])
    mf_agg = reses.groupby(['model', 'fold']).apply(lambda x: get_rsquared(var, x)).reset_index(name='r-squared')
    mf_agg['rmse'] = np.sqrt(reses.groupby(['model', 'fold']).squared_error.mean()).reset_index(name='rmse').loc[:, ['rmse']]
    # deal with grouping by subject
    mf_agg['rmse_unweighted'] = np.sqrt(reses.groupby(['model', 'fold', 'SDAN']).squared_error.mean().reset_index(name='rmse_unweighted').drop('SDAN', axis=1).groupby(['model', 'fold']).mean()).loc[:, 'rmse_unweighted'].values

    mf_agg['mae'] = reses.groupby(['model', 'fold']).abs_error.mean().reset_index(name='mae').loc[:, ['mae']]
    mf_agg['mae_unweighted'] = reses.groupby(['model', 'fold', 'SDAN']).abs_error.mean().reset_index(name='mae_unweighted').drop('SDAN', axis=1).groupby(['model', 'fold']).mean().loc[:, 'mae_unweighted'].values

    # and run bootstraps
    mfq_pw_res_path = Path(f'../data/mfq_sims/mfq_pairwise_res_sim-{sim_number}_fhtb-{fh_tb}.csv')
    pairwise_res = foldwise_bootstraps(mf_agg, None)
    pairwise_res['sim_number'] = sim_number
    pairwise_res['fh_tb'] = fh_tb
    pairwise_res.to_csv(mfq_pw_res_path, index=None)
    pw_reses.append(pairwise_res)

Formula: s_mfq_tot~antidepressants+TimeBetween+InpatientDuring+Age_at_visit+SEX+OtherMeds+postpandemic+(TimeBetween|SDAN)

Family: gaussian	 Inference: parametric

Number of observations: 1147	 Groups: {'SDAN': 115.0}

Log-likelihood: -3557.664 	 AIC: 7115.328

Random effects:

                 Name     Var    Std
SDAN      (Intercept)  19.708  4.439
SDAN      TimeBetween  28.166  5.307
Residual               23.246  4.821

              IV1          IV2   Corr
SDAN  (Intercept)  TimeBetween -0.349

Fixed effects:

Formula: s_mfq_tot~antidepressants+TimeBetween+InpatientDuring+Age_at_visit+SEX+OtherMeds+postpandemic+(TimeBetween|SDAN)

Family: gaussian	 Inference: parametric

Number of observations: 1147	 Groups: {'SDAN': 114.0}

Log-likelihood: -3534.237 	 AIC: 7068.473

Random effects:

                 Name     Var    Std
SDAN      (Intercept)  20.038  4.476
SDAN      TimeBetween  16.893  4.110
Residual               22.577  4.752

              IV1          IV2   Corr
SDAN  (Interc

In [76]:
mf_agg

,model,fold,r-squared,rmse,rmse_unweighted,mae,mae_unweighted
0,FH,0,0.071391,5.986515,6.260516,4.748065,5.134405
1,FH,1,-0.017491,6.797470,7.019385,5.441651,5.522082
2,FH,2,0.040718,6.759969,6.795436,5.510312,5.620986
3,FH,3,0.062450,6.518409,6.934372,5.136306,5.576044
4,FH,4,-0.212472,6.483499,6.205162,5.282152,5.128882
5,FH,5,-0.199068,6.285671,6.608896,5.102762,5.305681
6,FH,6,-0.163316,7.017983,6.684071,5.387526,5.013283
7,FH,7,0.083721,5.820676,5.610059,4.706286,4.593119
8,Null,0,0.077713,5.966101,6.430865,4.756175,5.297698
9,Null,1,-0.063412,6.949168,7.072461,5.571366,5.564093


In [57]:
tweaked = simmer.tweak_term({'fe__dep_immed:TimeBetween':-10})
mdl = Lmer(model, data=tweaked)
mdl_fit = mdl.fit(old_optimizer=True)
display(mdl_fit)
red_mdl = Lmer(nointeraction, data=tweaked)
red_mdl_fit = red_mdl.fit(old_optimizer=True)
display(red_mdl_fit)
y = tweaked.s_mfq_tot
f2 = fsquared(y, mdl.residuals, red_mdl.residuals)

Formula: s_mfq_tot~dep_immed*TimeBetween+MFQtminus1+InpatientDuring+Age_at_visit+SEX+antidepressants+OtherMeds+postpandemic+(TimeBetween|SDAN)

Family: gaussian	 Inference: parametric

Number of observations: 1311	 Groups: {'SDAN': 130.0}

Log-likelihood: -3832.489 	 AIC: 7664.977

Random effects:

                 Name     Var    Std
SDAN      (Intercept)  11.602  3.406
SDAN      TimeBetween  22.119  4.703
Residual               16.511  4.063

              IV1          IV2   Corr
SDAN  (Intercept)  TimeBetween -0.429

Fixed effects:



,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),7.900,2.603,13.198,2.703,237.844,2.923,0.004,**
dep_immed,0.913,-0.687,2.513,0.816,117.113,1.118,0.266,
TimeBetween,-1.194,-4.094,1.706,1.480,77.383,-0.807,0.422,
MFQtminus1,0.583,0.535,0.632,0.025,1246.848,23.488,0.000,***
InpatientDuring,-1.089,-2.037,-0.140,0.484,1104.269,-2.250,0.025,*
Age_at_visit,-0.270,-0.583,0.043,0.160,234.515,-1.690,0.092,.
SEX,0.988,-0.424,2.400,0.720,119.326,1.371,0.173,
antidepressants,0.770,-0.640,2.180,0.719,118.170,1.070,0.287,
OtherMeds,1.275,-0.298,2.848,0.803,116.059,1.589,0.115,
postpandemic,-0.177,-0.898,0.543,0.368,839.099,-0.482,0.630,


Formula: s_mfq_tot~dep_immed+TimeBetween+MFQtminus1+InpatientDuring+Age_at_visit+SEX+antidepressants+OtherMeds+postpandemic+(TimeBetween|SDAN)

Family: gaussian	 Inference: parametric

Number of observations: 1311	 Groups: {'SDAN': 130.0}

Log-likelihood: -3847.179 	 AIC: 7694.358

Random effects:

                 Name     Var    Std
SDAN      (Intercept)  12.460  3.530
SDAN      TimeBetween  37.035  6.086
Residual               16.538  4.067

              IV1          IV2   Corr
SDAN  (Intercept)  TimeBetween -0.481

Fixed effects:



,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),9.435,4.137,14.733,2.703,233.367,3.490,0.001,***
dep_immed,-1.298,-2.680,0.083,0.705,117.018,-1.842,0.068,.
TimeBetween,-8.128,-9.836,-6.419,0.872,97.591,-9.322,0.000,***
MFQtminus1,0.583,0.534,0.633,0.025,1241.384,23.302,0.000,***
InpatientDuring,-1.054,-2.017,-0.092,0.491,1160.625,-2.146,0.032,*
Age_at_visit,-0.269,-0.584,0.046,0.161,233.793,-1.675,0.095,.
SEX,1.115,-0.305,2.535,0.725,119.131,1.538,0.127,
antidepressants,0.833,-0.585,2.251,0.723,118.394,1.152,0.252,
OtherMeds,1.241,-0.339,2.821,0.806,116.307,1.539,0.126,
postpandemic,-0.247,-0.977,0.482,0.372,842.609,-0.664,0.507,


In [58]:
f2

-0.007650089280357288

In [34]:
tweaked.

,dep_immed,TimeBetween,MFQtminus1,InpatientDuring,Age_at_visit,SEX,antidepressants,OtherMeds,postpandemic,SDAN,s_mfq_tot
0,1,3,20.0,0,17.32,1,1,1,1,24328,25.202272
1,1,3,15.0,0,17.01,1,1,1,1,24311,10.724160
2,1,3,18.0,0,17.07,1,1,1,1,24311,12.615306
3,1,3,11.0,0,17.33,1,1,1,1,24311,14.232179
4,1,3,8.0,0,17.39,1,1,1,1,24311,10.964031
...,...,...,...,...,...,...,...,...,...,...,...
1306,0,3,5.0,0,18.92,0,1,0,1,23765,27.759913
1307,0,3,2.0,0,19.20,0,1,0,1,23765,27.569417
1308,0,3,11.0,0,19.42,0,1,0,1,23765,32.875377
1309,0,3,0.0,0,19.54,0,1,0,1,23765,25.073672


In [32]:
mdl_fit

,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),8.869,-1.330,19.069,5.204,570.114,1.704,0.089,.
dep_immed,2.301,-2.214,6.816,2.304,123.186,0.999,0.320,
MFQtminus1,0.587,0.532,0.643,0.028,1222.373,20.735,0.000,***
InpatientDuring,-2.318,-3.312,-1.324,0.507,1225.040,-4.570,0.000,***
Age_at_visit,0.366,-0.183,0.916,0.280,1088.253,1.307,0.192,
SEX,-1.631,-6.246,2.985,2.355,123.708,-0.692,0.490,
antidepressants,2.564,-2.082,7.209,2.370,123.050,1.082,0.282,
OtherMeds,-3.468,-8.685,1.748,2.662,122.767,-1.303,0.195,
postpandemic,-1.075,-2.048,-0.102,0.496,1266.015,-2.165,0.031,*


Formula: s_mfq_tot~dep_immed+TimeBetween+MFQtminus1+InpatientDuring+Age_at_visit+SEX+antidepressants+OtherMeds+postpandemic+(TimeBetween|SDAN)

Family: gaussian	 Inference: parametric

Number of observations: 1311	 Groups: {'SDAN': 130.0}

Log-likelihood: -3846.079 	 AIC: 7692.159

Random effects:

                 Name     Var    Std
SDAN      (Intercept)  15.910  3.989
SDAN      TimeBetween  17.864  4.227
Residual               16.603  4.075

              IV1          IV2   Corr
SDAN  (Intercept)  TimeBetween -0.539

Fixed effects:



,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),6.742,1.061,12.423,2.898,253.813,2.326,0.021,*
dep_immed,0.313,-1.216,1.842,0.780,117.464,0.401,0.689,
TimeBetween,-1.871,-3.327,-0.415,0.743,82.578,-2.519,0.014,*
MFQtminus1,0.600,0.551,0.649,0.025,1278.212,23.828,0.000,***
InpatientDuring,-0.871,-1.827,0.085,0.488,1130.186,-1.785,0.074,.
Age_at_visit,-0.183,-0.520,0.155,0.172,260.927,-1.061,0.290,
SEX,1.149,-0.417,2.715,0.799,119.509,1.438,0.153,
antidepressants,-1.060,-2.627,0.507,0.800,117.987,-1.326,0.188,
OtherMeds,0.640,-1.109,2.389,0.892,115.652,0.717,0.475,
postpandemic,-0.108,-0.849,0.632,0.378,816.129,-0.286,0.775,


In [63]:
pstats.rsquared(tweaked.s_mfq_tot, mdl.residuals)

0.7198422222433198

In [18]:
def fsquared(y, res_full, res_reduced):
    rsquared_full = pstats.rsquared(y, res_full)
    rsquared_reduced = pstats.rsquared(y, res_reduced)
    
    result = (rsquared_full - rsquared_reduced)/(1-rsquared_full)
    
    return result

In [ ]:
fsquared()